In [1]:
from pathlib import Path
import sys
from naver_cafe.setting import settings
from engine.sql.almaden import Sql
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time
import requests
import datetime
import pandas as pd
import pickle
from contextlib import suppress
import re
from html import unescape
db = Sql("datacast2")

In [2]:
def login(driver=None,username=None,password=None):
    driver.find_element_by_xpath('//*[@id="gnb_login_button"]').click()
    driver.find_element_by_xpath('//*[@id="id"]').send_keys(username)
    driver.find_element_by_xpath('//*[@id="pw"]').send_keys(password)
    driver.find_element_by_xpath('//*[@id="log.login"]').click()

def initiate_setting(task_keyword):
    setting_info = None
    try:
        setting_info = list(filter(lambda x : task_keyword in x.keys(),settings.info_list))[0][task_keyword]
    except Exception:
        print("task keyword 가 잘못되었습니다.")
    return setting_info

def select_task_id(task_keyword):
    row = db.select("crawl_task",'task_id','keyword=\"%s\" and channel=\"%s\"'%(task_keyword,"navercafe"))
    if len(row)>0:
        task_id = row[0]["task_id"]
    else:
        raise ValueError("task_id 에 해당하는 값이 없습니다.")
    return task_id

In [3]:
#setting
task_keyword = "라이즈오브킹덤즈"  ###crawl_task 에서의 키워드
setting_info = initiate_setting(task_keyword)
task_id = select_task_id(task_keyword)
print(setting_info,task_id)
username = setting_info["username"]
password = setting_info["password"]
page_link = setting_info["link"]
login_required = setting_info["login_required"]
# 카페 정보
cafe = {'name': task_keyword                          # 카페 이름
       ,'page_link': page_link}

{'login_required': True, 'username': 'vasana12@naver.com', 'password': 'xmrwodud01!', 'link': 'https://cafe.naver.com/riseofkingdoms'} 156251


In [4]:
### 크롬 드라이버 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--dns-prefetch-disable")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

### 카페 url 로 이동
driver.get(cafe.get("page_link"))



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\wodud\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [5]:
###카페 전체게시글 가져오기
WebDriverWait(driver,3).until(EC.presence_of_element_located((By.ID,'menuLink0')))
driver.find_element(By.XPATH,'//*[@id="menuLink0"]').click()
driver.implicitly_wait(0.5)
driver.switch_to.frame('cafe_main')


In [6]:
### 키워드 수집 정보
num_per_page = 15          # 페이지당 게시글 갯수(default: 15개)
address_list=[]
page = 1

l=True
while l:
    time.sleep(0.5)
    ### 현재 페이지의 html 불러오기
    r=None
    try:
        r = driver.page_source
    except TimeoutException as ex:
        print('TimeoutException',ex)
        driver.navigate().refresh()

    page_html = BeautifulSoup(r, "html.parser")
    #         로그인 안한 경우 아래로 진행

    content = page_html.find_all("div", class_="article-board m-tcol-c")[1].find('tbody')
    if content == None:
        content = page_html.find("div", class_="article-board result-board m-tcol-c").find('tbody')
    body = content.find_all("tr")
    ### 게시글 정보 저장하기
    for x in body:
        temp_dict={}
        if x.find("div", class_="board-list") is not None:
            #<a class="article" href="/ArticleRead.nhn?clubid=25474288&amp;page=1&amp;boardtype=L&amp;articleid=235073&amp;referrerAllArticles=true" onclick="clickcr(this, 'cfa.atitle','','',event);">

            temp_dict['link'] = "https://cafe.naver.com" + x.find('a',class_="article").get('href')
            temp_dict['name'] = x.find("td", class_="td_name").find('a',class_='m-tcol-c').text.strip()

            try:
                temp_dict['date'] = x.find("td", class_="td_date").text.strip().replace(".","-")[0:-1]
                temp_dict['date'] = datetime.datetime.strptime(temp_dict['date'],"%Y-%m-%d")
            except:
                temp_dict['date'] = datetime.datetime.now().date()
            temp_dict['date'] = datetime.datetime.strftime(temp_dict['date'],"%Y-%m-%d")
            temp_dict['view'] = x.find("td", class_="td_view").text.strip().replace(",","")
            if "만" in temp_dict['view']:
                temp_dict['view'] = temp_dict['view'].replace("만","")
                temp_dict['view'] = temp_dict['view'].replace(".","")
                temp_dict['view'] = int(temp_dict['view'])*10000
            else :
                temp_dict['view'] = int(temp_dict['view'])
            db.insert("crawl_contents",
                      task_id= task_id,
                      author = temp_dict['name'],
                      n_view = int(temp_dict['view']),
                      post_date = temp_dict['date'],
                      url=temp_dict['link'],
                      crawl_status = 'F')
    print("(현재시각) "+str(datetime.datetime.now())+": "+ str(page) +"page done")

    ### 다음 페이지로 넘어가기
    page+=1
    driver.implicitly_wait(1)
    try:
        if page<=10:   # 1~10 : 페이지 번호 그대로
            page_xpath = str(page)
            driver.find_element(By.XPATH,'//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
            # driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
        elif page == 11:   # 11 : 다음 버튼
            driver.find_element(By.XPATH,'//*[@id="main-area"]/div[7]/a[11]/span').click()
            # driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[11]/span').click()
        elif page>11 and page%10!=1:   # 12~ : 페이지 번호 마지막 자리 + 1
            page_xpath = str(page-((page-1)//10)*10+1)
            driver.find_element(By.XPATH,'//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
            # driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
        elif page%10 == 1:   # 21,31.. : 다음 버튼
            driver.find_element(By.XPATH, '//*[@id="main-area"]/div[7]/a[12]/span').click()
            # driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[12]/span').click()
    except Exception as e:
            print(e)
            l=False

(현재시각) 2022-02-21 13:54:35.804942: 1page done
(현재시각) 2022-02-21 13:54:37.762709: 2page done
(현재시각) 2022-02-21 13:54:39.803254: 3page done
(현재시각) 2022-02-21 13:54:41.557307: 4page done
(현재시각) 2022-02-21 13:54:43.512472: 5page done
(현재시각) 2022-02-21 13:54:45.299694: 6page done
(현재시각) 2022-02-21 13:54:46.884372: 7page done
(현재시각) 2022-02-21 13:54:48.648572: 8page done
(현재시각) 2022-02-21 13:54:50.724273: 9page done
(현재시각) 2022-02-21 13:54:52.423729: 10page done
(현재시각) 2022-02-21 13:54:54.440338: 11page done
(현재시각) 2022-02-21 13:54:56.136985: 12page done
(현재시각) 2022-02-21 13:54:57.940164: 13page done
(현재시각) 2022-02-21 13:54:59.792213: 14page done
(현재시각) 2022-02-21 13:55:01.304171: 15page done
(현재시각) 2022-02-21 13:55:02.928829: 16page done
(현재시각) 2022-02-21 13:55:04.574489: 17page done
(현재시각) 2022-02-21 13:55:06.384541: 18page done
(현재시각) 2022-02-21 13:55:08.132544: 19page done
(현재시각) 2022-02-21 13:55:09.784128: 20page done
(현재시각) 2022-02-21 13:55:11.476646: 21page done
(현재시각) 2022-02-21 13:5

WebDriverException: Message: chrome not reachable
  (Session info: chrome=98.0.4758.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A369A3+2582947]
	Ordinal0 [0x009CA6D1+2139857]
	Ordinal0 [0x008C3960+1063264]
	Ordinal0 [0x008B8092+1015954]
	Ordinal0 [0x008B88C8+1018056]
	Ordinal0 [0x008BA102+1024258]
	Ordinal0 [0x008B3DE9+998889]
	Ordinal0 [0x008C4DD0+1068496]
	Ordinal0 [0x00917B12+1407762]
	Ordinal0 [0x00908366+1344358]
	Ordinal0 [0x008E5176+1200502]
	Ordinal0 [0x008E6066+1204326]
	GetHandleVerifier [0x00BDBE02+1675858]
	GetHandleVerifier [0x00C9036C+2414524]
	GetHandleVerifier [0x00ACBB01+560977]
	GetHandleVerifier [0x00ACA8D3+556323]
	Ordinal0 [0x009D020E+2163214]
	Ordinal0 [0x009D5078+2183288]
	Ordinal0 [0x009D51C0+2183616]
	Ordinal0 [0x009DEE1C+2223644]
	BaseThreadInitThunk [0x761BFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77EB7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77EB7A6E+238]


In [ ]:
driver.quit()